In [ ]:
import math
import model as ty
import scipy.optimize as optim

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import heat_transfer as bht

In [ ]:
a = 0.001
delta = 0.00335

L_a = 0.050
N_ail = 75
k = 226

L_riser = 6*0.180

Q_f0 = -5.3/30

DT = -5.59

gam = Q_f0/(k*a*N_ail*delta*DT) 

In [ ]:
def gamma_int(a,k,h,delta,L_a,N_ail,L_riser):
    Bi = ty.Biot(a,k,h,delta)

    alpha = math.sqrt(2*Bi)
    beta = math.sqrt(Bi/2)*(1/(1+a/delta))

    arg = (alpha*L_a)/a
    numerateur = (alpha/a)*math.sinh(arg) + ((beta*alpha)/a)*math.cosh(arg)
    denominateur = math.cosh(arg) + beta*math.sinh(arg)

    gamma = numerateur/denominateur

    gamma_int = (k*a*N_ail*delta)*gamma/L_riser

    return gamma_int

In [ ]:
def gamma_val(h):
    Bi = ty.Biot(a,k,h,delta)

    alpha = math.sqrt(2*Bi)
    beta = math.sqrt(Bi/2)*(1/(1+a/delta))

    arg = (alpha*L_a)/a
    numerateur = (alpha/a)*math.sinh(arg) + ((beta*alpha)/a)*math.cosh(arg)
    denominateur = math.cosh(arg) + beta*math.sinh(arg)

    gamma = numerateur/denominateur

    gamma_int = (k*a*N_ail*delta)*gamma/L_riser

    gamma_back = delta/(1/(3.+h))

    return L_riser*(gamma_back+gamma_int)*DT - Q_f0

In [ ]:
delta/(1/(1.4+3.))

In [ ]:
gamma_int(a,k,1.+1.4,delta,L_a,N_ail,L_riser)

In [ ]:
optim.fsolve(gamma_val,2.)

In [ ]:
# Define T_abs_list, T_amb and DT_list and create df and pth

T_abs_list = np.array([15,17,19,21,23,25,27,29,31,33,35])
T_abs_list = T_abs_list + np.array(len(T_abs_list)*[273.15])

T_amb = 25+273.15

DT_list = []

for i in range(len(T_abs_list)):
    DT_list.append(T_abs_list[i]-T_amb)

df = pd.DataFrame(index=DT_list)
pth = pd.DataFrame(index=DT_list)

In [ ]:
# Compute heat transfer coefficient for flat plates

for i in range(len(T_abs_list)):
    df.loc[DT_list[i],'Flat plate facing downward'] = bht.back_h_simple(T_abs_list[i],T_amb,45,1.6)
    df.loc[DT_list[i],'Flat plate facing upward'] = bht.top_h_simple(T_abs_list[i],T_amb,45,1.6)

In [ ]:
# Plot df

plt.scatter(DT_list,df['Flat plate facing downward'],label="Facing downward")
plt.scatter(DT_list,df['Flat plate facing upward'],label="Facing upward")
plt.plot(DT_list,df['Flat plate facing downward'],"--")
plt.plot(DT_list,df['Flat plate facing upward'],"--")

for i in range(len(df['Flat plate facing downward'])):
    plt.annotate(round(df['Flat plate facing downward'].iloc[i],1), (DT_list[i], df['Flat plate facing downward'].iloc[i]))
    plt.annotate(round(df['Flat plate facing upward'].iloc[i],1), (DT_list[i], df['Flat plate facing upward'].iloc[i]))

plt.scatter([-6],[2.2],label='EXTIA',c='grey')
plt.annotate(2.2,(-6,2.2))

plt.xticks(DT_list)
plt.xlabel('T_plate_mean - T_amb (K)')
plt.ylabel('External heat transfer coefficient (m2K/W)')
plt.legend()
plt.show()

In [ ]:
# Compute heat transfer coefficients for fins

theta = 45.
longueur = 1.55
D_list = [0.001,0.005,0.0075,0.010,0.015,0.020,0.025,0.030,0.035,0.040,0.050,0.060,0.1,0.5]
L_fin = 0.020

for i in range(len(D_list)):

    for j in range(len(T_abs_list)):
        if j<=5:
            df.loc[DT_list[j],'Fins D = '+str(D_list[i])] = bht.back_h_fins(T_abs_list[j],T_amb,theta,longueur,D_list[i],L_fin)
        else:
            pass

In [ ]:
# list_col = np.array(df.columns)
list_col = ['Flat plate facing downward']+['Fins D = '+str(s) for s in [0.01,0.015,0.02,0.04,0.05,0.06,0.1,0.5]]

for k in range(len(list_col)):
    col = df[list_col[k]]
    plt.scatter(DT_list,col,label=col.name,marker='x')
    plt.plot(DT_list,col,"--")

    # for i in range(len(col)):
    #     plt.annotate(round(col.iloc[i],1), (DT_list[i], col.iloc[i]))

plt.scatter([-6],[1.23],label='EXTIA',c='grey')
plt.annotate(1.23,(-6,1.23))

plt.xlabel('T_plate_mean - T_amb (K)')
plt.ylabel('External heat transfer coefficient (m2K/W)')
plt.legend()
plt.show()

In [ ]:
pth

In [ ]:
# Compute thermal power for fins

for i in range(len(D_list)):

    N_ail = (D_list[i]+300*delta)/(D_list[i]+a)

    for j in range(len(T_abs_list)):

        flat = 300*L_riser*df.loc[DT_list[j],'Fins D = '+str(D_list[i])]*delta*DT_list[j]
        pth.loc[DT_list[j],'Pth fins D = '+str(D_list[i])] = flat + 300*L_riser*gamma_int(a,k,df.loc[DT_list[j],'Fins D = '+str(D_list[i])],delta,L_a,N_ail,L_riser)*DT_list[j]

In [ ]:
list_col = np.array(pth.columns)
print(list_col)

In [ ]:
list_col = np.array(pth.columns)
# list_col = ['Pth fins D = 0.001','Pth fins D = 0.005','Pth fins D = 0.0075','Pth fins D = 0.01','Pth fins D = 0.015','Pth fins D = 0.02']

for k in range(len(list_col)):
    col = pth[list_col[k]]
    plt.scatter(DT_list,col,label=col.name,marker='x')
    plt.plot(DT_list,col,"--")

    # for i in range(len(col)):
    #     plt.annotate(round(col.iloc[i],1), (DT_list[i], col.iloc[i]))

plt.xlabel('T_plate_mean - T_amb (K)')
plt.ylabel('Power (W)')
plt.legend()
plt.show()

In [ ]:
pth

In [ ]:
-pth.loc[-6].iloc[0]

In [ ]:
plt.scatter(D_list,-pth.loc[-6])
plt.plot(D_list,-pth.loc[-6],'--')

for i in range(len(D_list)):
    plt.annotate(round(-pth.loc[-6].iloc[i]), (D_list[i], -pth.loc[-6].iloc[i]))

plt.xlabel('Spacing D (m)')
plt.ylabel('Power (W)')

In [ ]:
D_list

In [ ]:
3.196*L_riser*300*delta*6

In [ ]:
300*L_riser*1.7*delta*6+35

In [ ]:
48-25

In [ ]:
for i in range(len(D_list)):

    

    for j in range(len(T_abs_list)):



        df.loc[DT_list[j],'Fins D = '+str(D_list[i])] = bht.back_h_fins(T_abs_list[j],T_amb,theta,longueur,D_list[i],L_fin)

In [ ]:
0.9*5.67*1E-8*(273.15*2+0.5+6.5)*((273.15+0.5)**2+(273.15+6.5)**2)